# Этап 7: Pruning (Прунинг — прореживание нейросети)

Если SVD сжимает матрицы, изменяя их структуру, то **Прунинг** просто «выключает» лишние связи. 

### Основная идея:
Многие веса в обученной модели близки к нулю и почти не влияют на результат. Если мы их обнулим, мы получим **разреженную (sparse)** матрицу. 

**Magnitude-based Pruning** работает по принципу:
1. Выбираем порог или процент весов (например, 30%).
2. Находим веса с наименьшим абсолютным значением $|w|$.
3. Приравниваем их к нулю.

Итоговый вес: 
$$w_{pruned} = 
\begin{cases} 
w, & \text{if } |w| > \text{threshold} \\
0, & \text{otherwise}
\end{cases}$$

In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
from src.model import GPTLanguageModel, device, estimate_loss, decode, encode
import copy

# 1. Загружаем оригинал
model = GPTLanguageModel().to(device)
model.load_state_dict(torch.load('model_ckpt.pt', map_location=device))
model.eval()

def get_val_loss(mdl):
    with torch.no_grad():
        return estimate_loss(mdl)['val'].item()

baseline_loss = get_val_loss(model)
print(f"Baseline Loss (FP32): {baseline_loss:.4f}")

## 2. Глобальный прунинг
Мы применим «глобальный» прунинг к модели. Это значит, что мы будем удалять 30% самых слабых связей во всей сети сразу, позволяя алгоритму самому решить, в каких слоях веса важнее.

In [ ]:
model_pruned = copy.deepcopy(model)

# Список слоев, которые мы хотим проредить (обычно это линейные слои)
parameters_to_prune = []
for name, module in model_pruned.named_modules():
    if isinstance(module, nn.Linear):
        parameters_to_prune.append((module, 'weight'))

# Применяем глобальный прунинг (удаляем 40% весов)
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.4, # 40% связей будет обнулено
 British English)
)

print("✅ 40% весов во всех линейных слоях обнулены!")

## 3. Анализ разреженности (Sparsity)
Проверим, сколько реально нулей появилось в матрицах.

In [ ]:
def check_sparsity(mdl):
    total_zeros = 0
    total_elements = 0
    for name, buffer in mdl.named_buffers():
        if 'weight_mask' in name:
            total_zeros += torch.sum(buffer == 0).item()
            total_elements += buffer.nelement()
    
    print(f"Общая разреженность модели: {100. * total_zeros / total_elements:.2f}%")

check_sparsity(model_pruned)

pruned_loss = get_val_loss(model_pruned)
print(f"Loss после прунинга (40%): {pruned_loss:.4f}")
print(f"Деградация: {pruned_loss - baseline_loss:.4f}")

## 4. Почему прунинг не уменьшает файл автоматически?
Важный момент: PyTorch просто заменяет веса на нули, но не удаляет их физически. Чтобы модель стала занимать меньше места на диске, её нужно сохранять в специальном разреженном формате или запаковать (например, через `.zip`).

### Проверим эффект на генерации текста

In [ ]:
print("--- [Pruned Model Output (40% Sparse)] ---")
context = torch.tensor(encode("ROMEO: "), dtype=torch.long, device=device).unsqueeze(0)
print(decode(model_pruned.generate(context, max_new_tokens=150)[0].tolist()))

## 5. Итеративный прунинг и Fine-tuning
В реальности после прунинга всегда делают **Fine-tuning**, чтобы оставшиеся веса «подхватили» функции удаленных. 

**Ваш челлендж**: Попробуйте запустить дообучение модели из Step 5, но на прореженной модели. Вы удивитесь, насколько быстро она восстановит качество даже с 50% удаленных весов!